In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch
!pip install transformers
!pip install datasets
!pip install trl
!pip install peft
!pip install numpy
!pip install pandas
!pip install tqdm


In [ ]:
!pip install evaluate

In [ ]:
import torch

from transformers import AutoModelForSequenceClassification, AutoTokenizer, T5Tokenizer, T5ForConditionalGeneration

from torch.utils.data import DataLoader, Dataset as TorchDataset
from torch.optim import AdamW

from datasets import load_dataset, Dataset as HFDataset

from peft import PeftModel, PeftConfig,  TaskType

from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    PeftType,
    LoraConfig,
)

import trl
from trl import PPOTrainer, PPOConfig, AutoModelForSeq2SeqLMWithValueHead
from trl import create_reference_model
from trl.core import LengthSampler

import evaluate

import numpy as np
import pandas as pd


from tqdm import tqdm
tqdm.pandas()


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [ ]:

reward_model_directory = "/content/drive/MyDrive/BigData/models/step1RewardModel.pth"

rm_model = AutoModelForSequenceClassification.from_pretrained(reward_model_directory)
rm_tokenizer = AutoTokenizer.from_pretrained(reward_model_directory)
rm_model.to(device)


Some weights of the model checkpoint at /content/drive/MyDrive/BigData/models/step1RewardModel.pth were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.1.attention.self.value.lora_B.default.weight', 'bert.encoder.layer.5.attention.self.value.lora_B.default.weight', 'bert.encoder.layer.1.attention.self.query.lora_A.default.weight', 'bert.encoder.layer.3.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.7.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.4.attention.self.query.lora_B.default.weight', 'bert.encoder.layer.11.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.10.attention.self.query.lora_B.default.weight', 'classifier.modules_to_save.default.weight', 'bert.encoder.layer.6.attention.self.value.lora_A.default.weight', 'bert.encoder.layer.7.attention.self.query.lora_A.default.weight', 'bert.encoder.layer.2.attention.self.query.lora_B.default.weight', 'classifier.modules_to_save.default.bias', 'classifie

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
import torch.nn.functional as F


def score_summaries(model, tokenizer, chosen_summary, rejected_summary):
    # Tokenize the inputs
    chosen_tokens = tokenizer(chosen_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
    rejected_tokens = tokenizer(rejected_summary, return_tensors="pt", padding='max_length', truncation=True, max_length=512)

    chosen_tokens.to(device)
    rejected_tokens.to(device)

    # Get logits from the model
    with torch.no_grad():
        chosen_logits = model(**chosen_tokens).logits
        rejected_logits = model(**rejected_tokens).logits

    # print("chosen logits : ", chosen_logits)
    # print("rejected logits : ", rejected_logits)
    # Apply softmax to get probabilities
    chosen_probs = F.softmax(chosen_logits, dim=-1)
    rejected_probs = F.softmax(rejected_logits, dim=-1)
    # print("chosen prob : ", chosen_probs)
    # print("rejected prob : ", rejected_probs)
    # Assuming the positive class (indicating 'chosen' is good) is the second one
    chosen_score = chosen_probs[0][1].item()
    rejected_score = rejected_probs[0][1].item()

    # Extract logits for each summary
    chosen_logit = chosen_logits[0][1].item()
    rejected_logit = rejected_logits[0][1].item()

    return chosen_score, rejected_score, chosen_logit, rejected_logit

In [ ]:

chosen_summary = "Water meter in another condo is not in our condo. What can we do legally to restore water to my condo complex?"
rejected_summary = "Go fix the problem."

chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")

Chosen Score: 0.5957
Rejected Score: 0.5422
Chosen Logit: 0.2633
Rejected Logit: 0.2005


In [ ]:
pip install sentencepiece

In [ ]:
policy_model_path = "JuanKO/rlhf_base_model"
policy_model_name = "t5-base"

policy_model = T5ForConditionalGeneration.from_pretrained(policy_model_path)
policy_model.to(device)
policy_tokenizer = T5Tokenizer.from_pretrained(policy_model_path)

tokenizer_config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
task_prefix = "Complete and elaborate: "

text = "I once prevented the rape of a woman by butt-stroking her attacker, "
#text = "SUBREDDIT: r/legaladvice TITLE: What can I do legally to restore water to my condominium!? POST: Hi, I live in SE Michigan in a condominium complex. Our water was shut off due to non-payment. (we recieved no notice) and we had to pay all that was due ($1500) We payed this yesterday at 2, they said the water would be turned on immediately. It wasn't. It's now the next day. The lady in our assosciation keeps insisting that the water meter is in another condo. Which we can't access because the person living there is never there (it's being rented) Now we're stuck with no water, no shower, no teeth brushing, no toilets, and no food for certain meals.... Please help us... What can we do? We called the police and they say that we can file a civil report for the lady not doing her job..."
prompt = f"{task_prefix}{text}"
input_ids = policy_tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = policy_model.generate(input_ids, max_length=100).to(device)

strOutput = policy_tokenizer.decode(outputs[0], skip_special_tokens=True)
print(strOutput)
chosen_score, rejected_score, chosen_logit, rejected_logit = score_summaries(rm_model, rm_tokenizer, strOutput, "")

print(f"Chosen Score: {chosen_score:.4f}")
print(f"Rejected Score: {rejected_score:.4f}")

print(f"Chosen Logit: {chosen_logit:.4f}")
print(f"Rejected Logit: {rejected_logit:.4f}")


: I once prevented the rape of a woman by butt-stroking her attacker.
Chosen Score: 0.5367
Rejected Score: 0.5651
Chosen Logit: 0.1904
Rejected Logit: 0.2405


In [ ]:
lora_config = LoraConfig(
    r=8, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.10,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # T5
)

policy_peft_model = get_peft_model(policy_model, lora_config)
policy_peft_model.to(device)

PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): T5ForConditionalGeneration(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): Linear(
                    in_features=768, out_features=768, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=768, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=768, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_embedding_B): Pa

In [ ]:
policy_peft_model.print_trainable_parameters()

trainable params: 884,736 || all params: 223,788,288 || trainable%: 0.3953450861557152


In [ ]:
# https://huggingface.co/docs/trl/quickstart
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(policy_peft_model,
                                                               torch_dtype=torch.bfloat16,
                                                               is_trainable=True)

ppo_model.to(device)

AutoModelForSeq2SeqLMWithValueHead(
  (pretrained_model): PeftModelForSeq2SeqLM(
    (base_model): LoraModel(
      (model): T5ForConditionalGeneration(
        (shared): Embedding(32128, 768)
        (encoder): T5Stack(
          (embed_tokens): Embedding(32128, 768)
          (block): ModuleList(
            (0): T5Block(
              (layer): ModuleList(
                (0): T5LayerSelfAttention(
                  (SelfAttention): T5Attention(
                    (q): Linear(
                      in_features=768, out_features=768, bias=False
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.1, inplace=False)
                      )
                      (lora_A): ModuleDict(
                        (default): Linear(in_features=768, out_features=8, bias=False)
                      )
                      (lora_B): ModuleDict(
                        (default): Linear(in_features=8, out_features=768, bias=False)
                      

In [ ]:
ref_model = create_reference_model(policy_model)
ref_model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(
                in_features=768, out_features=768, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=768, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=768, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(
                i

In [ ]:
from datasets import Dataset
import pandas as pd
dataset_5000= pd.read_csv('/content/drive/MyDrive/BigData/datasets/accepted_rejected_responses_5000.csv')
dataset_5000.rename(columns={'Prompt': 'prompt', 'Accepted_response': 'chosen', 'Rejected_response': 'rejected'}, inplace=True)
# Assuming `formatted_dataset` is a DataFrame after applying `formatting_func`
dataset_5000.head()

,Unnamed: 0,prompt,chosen,rejected
0,0,Corruption involving the contractors is the ch...,U.S. Supreme Court decision on alleged corrupt...,Corruption involving the contractors is the ch...
1,1,5][6][7] The term was coined by the author Maki,5][6][7] The term was coined by the author Mak...,Maki
2,2,"As for the internals, the people with the most...","As for the internals, the people with the most...",people with the most say are people with the m...
3,3,"On March 2, 2017, roughly 100 of our","On March 2, 2017, roughly 100 of our 2500 stud...",tens of thousands of people were killed in a c...
4,4,"Syrian Democratic Forcesâ€™ spokesman, Talal S...","Syrian Democratic Forces’ spokesman, Talal Sil...","Syrian Democratic Forces spokesman, Talal Silo..."


In [ ]:
orig_dataset = Dataset.from_pandas(dataset_5000)

In [ ]:
def formatting_func(examples):
    kwargs = {"padding": "max_length", "truncation": True, "max_length": 512, "return_tensors": "pt"}

    prompt_plus_chosen_response = examples["Accepted_response"]

    prompt_plus_rejected_response = examples["Rejected_response"]
    if(examples["Accepted_response"] is None):
      prompt_plus_chosen_response = str('')
    if(examples["Rejected_response"] is None):
      prompt_plus_rejected_response = str('')

    # print(prompt_plus_rejected_response)
    # tokens_chosen = tokenizer(prompt_plus_chosen_response, **kwargs)
    # tokens_rejected = tokenizer(prompt_plus_rejected_response, **kwargs)
    tokens_chosen = tokenizer.encode_plus(prompt_plus_chosen_response, **kwargs )
    tokens_rejected = tokenizer.encode_plus(prompt_plus_rejected_response, **kwargs)
    return {
        "input_ids_chosen": tokens_chosen["input_ids"][0], "attention_mask_chosen": tokens_chosen["attention_mask"][0],
        "input_ids_rejected": tokens_rejected["input_ids"][0], "attention_mask_rejected": tokens_rejected["attention_mask"][0]
    }


In [ ]:
formatted_dataset = orig_dataset.map(formatting_func)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

KeyError: ignored

In [ ]:
none_filtered = orig_dataset.filter(lambda example: example['prompt'] is not None)

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
len(none_filtered)

4974

In [ ]:

filtered_dataset = none_filtered.filter(lambda example: len(example['prompt'].split()) <= 450) # By word


shuffled_dataset = filtered_dataset.shuffle(seed=42).select(range(2000))


new_dataset_dict = {
    "prompt": shuffled_dataset["prompt"],
    "response": shuffled_dataset["chosen"]
}

dataset = HFDataset.from_dict(new_dataset_dict)


split_ratio = 0.8  # 80% for training, 20% for evaluation
num_train_samples = int(split_ratio * len(dataset))
train_dataset = dataset.select(range(num_train_samples))
eval_dataset = dataset.select(range(num_train_samples, len(dataset)))

Filter:   0%|          | 0/4974 [00:00<?, ? examples/s]

In [ ]:
print(train_dataset[0].keys())
print(eval_dataset[0].keys())

dict_keys(['prompt', 'response'])
dict_keys(['prompt', 'response'])


In [ ]:
print("train dataset len : ", len(train_dataset))
print("eval dataset len : ", len(eval_dataset))

train dataset len :  1600
eval dataset len :  400


In [ ]:
from transformers import T5Tokenizer

# Instantiate your tokenizer (replace T5Tokenizer with your model's tokenizer if different)
tokenizer = T5Tokenizer.from_pretrained("t5-small") # or whatever model you're using

def tokenize_function(example):
    # Tokenize the prompt and store it as input_ids. Also return the response.
    return {
        "input_ids": tokenizer(example["prompt"], return_tensors="pt", truncation=True, max_length=512)["input_ids"].squeeze(),
        "response": example["response"],
    }

# Tokenize the training and evaluation datasets
train_dataset = train_dataset.map(tokenize_function, batched=False)
eval_dataset = eval_dataset.map(tokenize_function, batched=False)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['prompt', 'response', 'input_ids'],
    num_rows: 1600
})

In [ ]:
# Lets check one sample of the train_dataset
print(train_dataset[0])  # print the first example from the training dataset

{'prompt': 'After a second-half that was dominated by the Hoosiers, out-shooting the Bruins by a 9-', 'response': 'The Syrian Electronic Army has claimed to be behind a number of high-profile cyber-attacks over recent years.', 'input_ids': [621, 3, 9, 511, 18, 17114, 24, 47, 3, 16032, 57, 8, 1546, 32, 7, 4518, 6, 91, 18, 5630, 32, 1222, 8, 272, 23162, 57, 3, 9, 668, 18, 1]}


In [ ]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

test_data = [{"key1": "value1", "key2": "value2", "key3": "value3"}, {"key1": "value4", "key2": "value5", "key3": "value6"}]
print(f'Collator input: {test_data}')
print(f'Collator output: {collator(test_data)}')

# Lets sample what the collator generates:
sample_data = [train_dataset[i] for i in range(3)]  # take first three examples
collated_data = collator(sample_data)
print(collated_data.keys())

learning_rate=1.41e-5
max_ppo_epochs=3
mini_batch_size=4
batch_size=16

# This is a HACK... lets see how this works out. May casue bias or may help. The good side is that this, being constant, can effect some type of regularization, preventing the model from gravitating too much towards any specific pattern in the training data.  Just a thought.
DEFAULT_REJECTED_SUMMARY_TEXT = "This is a bad response"

# Some initial values
output_min_length = 100
output_max_length = 400
output_length_sampler = LengthSampler(output_min_length, output_max_length)

# These hyperparams guide the generation of the completion in the policy model. We could add other params like temperature.
generation_kwargs = {
    "temperature": 1.0,
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True
}

max_ppo_steps = 100


Collator input: [{'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}, {'key1': 'value4', 'key2': 'value5', 'key3': 'value6'}]
Collator output: {'key1': ['value1', 'value4'], 'key2': ['value2', 'value5'], 'key3': ['value3', 'value6']}
dict_keys(['prompt', 'response', 'input_ids'])


In [ ]:
# Check out https://huggingface.co/docs/trl/trainer

config = PPOConfig(
    model_name=policy_model_name,
    learning_rate=learning_rate,
    ppo_epochs=max_ppo_epochs,
    mini_batch_size=mini_batch_size,
    batch_size=batch_size
)

In [ ]:

ppo_trainer = PPOTrainer(config = config,
                         model = ppo_model,
                         ref_model = ref_model,
                         tokenizer = policy_tokenizer,
                         dataset = train_dataset,
                         data_collator = collator)

In [ ]:
for step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    if step >= max_ppo_steps: # Break when we reach max_steps.
        break

    prompt_tensors = batch["input_ids"]

    if isinstance(prompt_tensors, list) and all(isinstance(item, list) for item in prompt_tensors): # HACK!!! Check if original_prompt_tensors is a list of lists
        lengths = [len(seq) for seq in prompt_tensors] # Verify if sequences have fixed or variable length
        unique_lengths = set(lengths)

        if len(unique_lengths) > 1: # If sequences have variable lengths, pad them
            max_length = max(unique_lengths)
            original_prompt_tensors = [seq + [0] * (max_length - len(seq)) for seq in prompt_tensors]  # padding with zeros

        prompt_tensors = [torch.tensor(seq).to(device) for seq in prompt_tensors] # Convert original_prompt_tensors to individual tensors

    summary_tensors = []

    for prompt_tensor in prompt_tensors:
        prompt_tensor = torch.tensor(prompt_tensor).to(device)
        max_new_tokens = output_length_sampler()
        generation_kwargs["max_new_tokens"] = max_new_tokens
        summary = ppo_trainer.generate(prompt_tensor, **generation_kwargs)
        summary_tensors.append(summary.squeeze()[-max_new_tokens:])

    batch["response"] = [policy_tokenizer.decode(r.squeeze()) for r in summary_tensors]

    chosen_summaries = batch["response"]
    rejected_summaries = [DEFAULT_REJECTED_SUMMARY_TEXT] * len(batch["response"])

    reward_tensors = []

    for chosen_summary, rejected_summary in zip(chosen_summaries, rejected_summaries):
        chosen_score, _, _, _ = score_summaries(rm_model, rm_tokenizer, chosen_summary, rejected_summary)
        reward_tensors.append(torch.tensor(chosen_score))

    stats = ppo_trainer.step(prompt_tensors, summary_tensors, reward_tensors)
    ppo_trainer.log_stats(stats, batch, reward_tensors)

    print(f'objective/kl: {stats["objective/kl"]}') # Measures how different the policy's action distribution after the update is from the action distribution before the update. PPO tries to make these changes very small to avoid sudden changes.
    print(f'ppo/returns/mean: {stats["ppo/returns/mean"]}') # This is the average return achieved by the agent. Higher is better.
    print(f'ppo/policy/advantages_mean: {stats["ppo/policy/advantages_mean"]}') # Measures how much better an action is than the average action at a given state.
    print('-'.join('' for x in range(100)))

0it [00:00, ?it/s]<ipython-input-31-da1b0f530615>:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  prompt_tensor = torch.tensor(prompt_tensor).to(device)
1it [00:16, 16.61s/it]

objective/kl: 0.0
ppo/returns/mean: 0.08307488262653351
ppo/policy/advantages_mean: 0.041002899408340454
---------------------------------------------------------------------------------------------------


2it [00:39, 20.11s/it]

objective/kl: 0.001664219656959176
ppo/returns/mean: 0.002474438399076462
ppo/policy/advantages_mean: 0.09100578725337982
---------------------------------------------------------------------------------------------------


3it [01:00, 20.52s/it]

...

objective/kl: -0.36683526635169983
ppo/returns/mean: 0.5022531747817993
ppo/policy/advantages_mean: 0.020124981179833412
---------------------------------------------------------------------------------------------------


99it [32:05, 17.16s/it]

objective/kl: 0.5114685297012329
ppo/returns/mean: 0.47951561212539673
ppo/policy/advantages_mean: 0.014324142597615719
---------------------------------------------------------------------------------------------------


100it [32:22, 19.43s/it]

objective/kl: 0.28421664237976074
ppo/returns/mean: 0.48302555084228516
ppo/policy/advantages_mean: -0.030520519241690636
---------------------------------------------------------------------------------------------------


In [ ]:
ppo_model_path = "/content/drive/MyDrive/BigData/models/fine_tuned_flan"

# Save the model
ppo_model.save_pretrained(ppo_model_path)

# Save the tokenizer
policy_tokenizer.save_pretrained(ppo_model_path)

('/content/drive/MyDrive/BigData/models/fine_tuned_flan/tokenizer_config.json',
 '/content/drive/MyDrive/BigData/models/fine_tuned_flan/special_tokens_map.json',
 '/content/drive/MyDrive/BigData/models/fine_tuned_flan/spiece.model',
 '/content/drive/MyDrive/BigData/models/fine_tuned_flan/added_tokens.json')

In [ ]:
ppo_saved_model_path = "/content/drive/MyDrive/BigData/models/fine_tuned_flan"

from trl import AutoModelForSeq2SeqLMWithValueHead # https://huggingface.co/docs/trl/quickstart
ppo_model = AutoModelForSeq2SeqLMWithValueHead.from_pretrained(ppo_saved_model_path)

from transformers import AutoTokenizer
policy_tokenizer = AutoTokenizer.from_pretrained(ppo_saved_model_path)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
ppo_model = ppo_model.to(device='cuda')


In [ ]:
def generate_summary(prompt: str, model, tokenizer, generation_kwargs, output_length_sampler) -> str:
    """
    Generate a summary for a given prompt using a trained policy model.

    Args:
    - prompt (str): The input text for which a summary needs to be generated.
    - model: The trained policy model.
    - tokenizer: The tokenizer used for the policy model.
    - generation_kwargs (dict): Arguments used for response generation.
    - output_length_sampler (func): Function to sample the length of the output.

    Returns:
    - str: Generated summary.
    """

    # Tokenize the prompt
    prompt_tensor = tokenizer.encode(prompt, return_tensors='pt').to(device)

    # Ensure it's only one tensor and check its shape
    assert prompt_tensor.dim() == 2, f"Unexpected tensor shape: {prompt_tensor.shape}"

    # Set the generation arguments
    max_new_tokens = output_length_sampler()
    generation_kwargs["max_new_tokens"] = max_new_tokens

    # Generate a summary
    summary_tensor = model.generate(input_ids=prompt_tensor, **generation_kwargs)

    # Decode and return the summary
    summary = tokenizer.decode(summary_tensor[0], skip_special_tokens=True)
    return summary


In [ ]:
# text = "SUBREDDIT: r/relationships TITLE: How do I/do I at all [20 F] tell my boyfriend [23 M] that I'm bisexual? POST: I've had two serious relationships prior to this one, both with women. They had no problem with me being bisexual and it was something known before the relationship -- my first girlfriend was also bisexual. I am now in a relationship with a guy. We've been exclusive for about a month. Having never faced this issue, I come to you, Reddit. Is this something that he needs to know? Is it really relevant to a hetero relationship, regardless of if one of the participants in the relationship is bisexual? If you guys think it is necessary, when do you think is the right time? I think my biggest fear is losing him because of it. I know that I should be with someone who is fine with who I am, but I really like the guy and I'd hate for my sexual orientation to be the thing that kills this."
# text = "SUBREDDIT: r/legaladvice TITLE: What can I do legally to restore water to my condominium!? POST: Hi, I live in SE Michigan in a condominium complex. Our water was shut off due to non-payment. (we recieved no notice) and we had to pay all that was due ($1500) We payed this yesterday at 2, they said the water would be turned on immediately. It wasn't. It's now the next day. The lady in our assosciation keeps insisting that the water meter is in another condo. Which we can't access because the person living there is never there (it's being rented) Now we're stuck with no water, no shower, no teeth brushing, no toilets, and no food for certain meals.... Please help us... What can we do? We called the police and they say that we can file a civil report for the lady not doing her job..."
# text = "SUBREDDIT: r/relationships TITLE: To go or not to go? Old friend (f, 23) getting married, I (f 23) don't want to because I have to go from here in the Netherlands to USA. POST: So, I have had this friend for a long time and we have always been there for each other. But about 6 months ago I moved here to the Netherlands to be with my partner (m23). This is our first place together here and we had to buy our own furniture. Needless to say we don't really have any money for trips. My friend is getting married in March in the USA and I feel really guilty out of obligation but I really don't want to go. I don't have the money for it and I don't want to leave here and miss my partner. Reasons for not wanting to go: 1. Money 2. Missing my partner. 3. Being incredibly bored once I'm there! I won't have a car or a way to get around, so I'll just be sitting in my parents house all day. I know it's bad that I don't want to go, but I am just really dreading it. Reddit, what do I do?"
# text = "SUBREDDIT: r/Advice TITLE: Bike tour around the world? POST: Hi there redditors! First of all I'd like to apologize for my English, but as you will see (I hope not), I'm not a native speaker. I'm 23-year-old who recently graduated from university and just stared my first job. Now, you see, my job is interesting and all, but it's an office job and I feel I'm not suited for this. I'm the adventures type, I want something happening around me and going to work from 9 to 6 is just killing me. The one thing that I thought of is a bike trip mostly in Europe, Asia and North Africa. The problem is that I'm from a country with an average salary around 350 euros or 450 USD. My salary is a bit higher - around 450 euros, but still not enough according to what I read is needed for such a trip, witch is about 30000 USD. My question is if somebody has done something like this without any money and if they have some tips for me. I'm thinking about sleeping outdoors or helping some locals for food and a place to crash. Is this something that could work out? I'm planning to go with my girlfriend and I think not too many people would take us in. Any help would be greatly appreciated!"
# text = "SUBREDDIT: r/Parenting TITLE: Question about saying 'no' to 18 month old POST: When I tell my son 'no' to something that is either dangerous (like sitting on the arm of the couch or trying to climb onto the television) or something that is an unwanted behavior (biting, hitting etc.) he looks at me and giggles before continuing to do whatever the hell he wants to do. When my husband tells him 'no' he stops what he's doing and sometimes gets upset to the point of crying (I think because his feelings are hurt). I guess the question is, how do I get him to listen to me and not just to his father? I have tried to make my voice sound louder and more masculine, but that just makes him laugh even harder."
# text = "Today Mr Turnbull admitted he was in danger of losing"
text = "That link is to the video, which unfortunately you√¢‚Ç¨‚Ñ¢ll actually"

In [ ]:
prompt = f"{task_prefix}{text}"
generated_summary = generate_summary(prompt, ppo_model, policy_tokenizer, generation_kwargs, output_length_sampler)
print(generated_summary)

: Absolutely video, for youtube part way. Now, here s how I did it.
